In [1]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive



Mounted at /gdrive
/gdrive


In [2]:
trainfile = r'/gdrive/MyDrive/CIS_508/IA_2/Insurance Fraud - TRAIN-3000.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/MyDrive/CIS_508/IA_2/Insurance Fraud -TEST-12900.csv'
testData = pd.read_csv(testfile)  #creates a dataframe


print(trainData.shape)
print(testData.shape)

trainData.head()
testData.head()

(2999, 32)
(12918, 32)


,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY,FRAUDFOUND
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision,Yes
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils,Yes
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils,Yes
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils,Yes
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision,Yes


In [3]:
#Data Description
trainData.info()
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   MONTH                 2999 non-null   object
 1   WEEKOFMONTH           2999 non-null   int64 
 2   DAYOFWEEK             2999 non-null   object
 3   MAKE                  2999 non-null   object
 4   ACCIDENTAREA          2999 non-null   object
 5   DAYOFWEEKCLAIMED      2999 non-null   object
 6   MONTHCLAIMED          2999 non-null   object
 7   WEEKOFMONTHCLAIMED    2999 non-null   int64 
 8   SEX                   2999 non-null   object
 9   MARITALSTATUS         2999 non-null   object
 10  AGE                   2999 non-null   int64 
 11  FAULT                 2999 non-null   object
 12  POLICYTYPE            2999 non-null   object
 13  VEHICLECATEGORY       2999 non-null   object
 14  VEHICLEPRICE          2999 non-null   object
 15  REPNUMBER             2999 non-null   

In [4]:
#Copy Train data excluding target
trainData_Copy = trainData.iloc[:, :-1].copy()
testData_Copy = testData.iloc[:, :-1].copy()

trainData_Copy.head()

,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision


In [7]:
#List of names of all Columns from a dataframe

TrainCols = list(trainData.columns.values)
TestCols = list(testData.columns.values)
print(TrainCols)
print(TestCols)

['MONTH', 'WEEKOFMONTH', 'DAYOFWEEK', 'MAKE', 'ACCIDENTAREA', 'DAYOFWEEKCLAIMED', 'MONTHCLAIMED', 'WEEKOFMONTHCLAIMED', 'SEX', 'MARITALSTATUS', 'AGE', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY', 'VEHICLEPRICE', 'REPNUMBER', 'DEDUCTIBLE', 'DRIVERRATING', 'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS', 'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED', 'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS', 'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'YEAR', 'BASEPOLICY', 'FRAUDFOUND']
['MONTH', 'WEEKOFMONTH', 'DAYOFWEEK', 'MAKE', 'ACCIDENTAREA', 'DAYOFWEEKCLAIMED', 'MONTHCLAIMED', 'WEEKOFMONTHCLAIMED', 'SEX', 'MARITALSTATUS', 'AGE', 'FAULT', 'POLICYTYPE', 'VEHICLECATEGORY', 'VEHICLEPRICE', 'REPNUMBER', 'DEDUCTIBLE', 'DRIVERRATING', 'DAYS_POLICY_ACCIDENT', 'DAYS_POLICY_CLAIM', 'PASTNUMBEROFCLAIMS', 'AGEOFVEHICLE', 'AGEOFPOLICYHOLDER', 'POLICEREPORTFILED', 'WITNESSPRESENT', 'AGENTTYPE', 'NUMBEROFSUPPLIMENTS', 'ADDRESSCHANGE_CLAIM', 'NUMBEROFCARS', 'YEAR', 'BASEPOLICY', 'F

In [8]:
#List of Categorical Features
categoricalFeatures = ["MONTH", "DAYOFWEEK", "MAKE", "ACCIDENTAREA",
                       "DAYOFWEEKCLAIMED", "MONTHCLAIMED", "SEX", "MARITALSTATUS",
                       "FAULT", "POLICYTYPE", "VEHICLECATEGORY", "VEHICLEPRICE",
                       "DAYS_POLICY_ACCIDENT", "DAYS_POLICY_CLAIM",
                       "PASTNUMBEROFCLAIMS", "AGEOFVEHICLE", "AGEOFPOLICYHOLDER",
                       "POLICEREPORTFILED", "WITNESSPRESENT", "AGENTTYPE",
                       "NUMBEROFSUPPLIMENTS", "ADDRESSCHANGE_CLAIM",
                       "NUMBEROFCARS", "BASEPOLICY"]

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy,testData_Copy], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
X_Train = combined_Data.xs(0)
X_Test = combined_Data.xs(1)

X_Test.head()


,WEEKOFMONTH,WEEKOFMONTHCLAIMED,AGE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,YEAR,MONTH_Apr,MONTH_Aug,MONTH_Dec,MONTH_Feb,MONTH_Jan,MONTH_Jul,MONTH_Jun,MONTH_Mar,MONTH_May,MONTH_Nov,MONTH_Oct,MONTH_Sep,DAYOFWEEK_Friday,DAYOFWEEK_Monday,DAYOFWEEK_Saturday,DAYOFWEEK_Sunday,DAYOFWEEK_Thursday,DAYOFWEEK_Tuesday,DAYOFWEEK_Wednesday,MAKE_Accura,MAKE_BMW,MAKE_Chevrolet,MAKE_Dodge,MAKE_Ferrari,MAKE_Ford,MAKE_Honda,MAKE_Jaguar,MAKE_Lexus,MAKE_Mazda,MAKE_Mecedes,MAKE_Mercury,MAKE_Nisson,MAKE_Pontiac,...,AGEOFVEHICLE_2_years,AGEOFVEHICLE_3_years,AGEOFVEHICLE_4_years,AGEOFVEHICLE_5_years,AGEOFVEHICLE_6_years,AGEOFVEHICLE_7_years,AGEOFVEHICLE_more_than_7,AGEOFVEHICLE_new,AGEOFPOLICYHOLDER_16_to_17,AGEOFPOLICYHOLDER_18_to_20,AGEOFPOLICYHOLDER_21_to_25,AGEOFPOLICYHOLDER_26_to_30,AGEOFPOLICYHOLDER_31_to_35,AGEOFPOLICYHOLDER_36_to_40,AGEOFPOLICYHOLDER_41_to_50,AGEOFPOLICYHOLDER_51_to_65,AGEOFPOLICYHOLDER_over_65,POLICEREPORTFILED_No,POLICEREPORTFILED_Yes,WITNESSPRESENT_No,WITNESSPRESENT_Yes,AGENTTYPE_External,AGENTTYPE_Internal,NUMBEROFSUPPLIMENTS_1_to_2,NUMBEROFSUPPLIMENTS_3_to_5,NUMBEROFSUPPLIMENTS_more_than_5,NUMBEROFSUPPLIMENTS_none,ADDRESSCHANGE_CLAIM_1_year,ADDRESSCHANGE_CLAIM_2_to_3_years,ADDRESSCHANGE_CLAIM_4_to_8_years,ADDRESSCHANGE_CLAIM_no_change,ADDRESSCHANGE_CLAIM_under_6_months,NUMBEROFCARS_1-vehicle,NUMBEROFCARS_2-vehicles,NUMBEROFCARS_3_to_4,NUMBEROFCARS_5_to_8,NUMBEROFCARS_more_than_8,BASEPOLICY_All_Perils,BASEPOLICY_Collision,BASEPOLICY_Liability
0,3,4,21,4,400,4,1994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
1,5,1,68,9,400,3,1994,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0
2,1,1,50,8,400,2,1994,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0
3,1,3,39,1,400,3,1994,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0
4,5,1,43,1,400,4,1994,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0


In [9]:
#Select Target Column and Tranform into Numerical variable
Y_Train = label_binarize(trainData.iloc[:, -1], classes=["No", "Yes"])
Y_Test = label_binarize(testData.iloc[:, -1], classes=["No", "Yes"])


In [10]:
#Decision Tree Classifier ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(X_Train, Y_Train)
clf_predict=clf.predict(X_Test)
print("accuracy Score (testset) for Decision Tree:{0:6f}".format(clf.score(X_Test,Y_Test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(Y_Test,clf_predict))


accuracy Score (testset) for Decision Tree:0.877613
Confusion Matrix for Decision Tree
[[10887  1533]
 [   48   450]]


In [ ]:
#print(clf_random.cv_results_)
#print(clf_grid.cv_results_)


=====


In [ ]:
print(clf.feature_importances_)

[0.03626163 0.02188311 0.07006032 0.04736395 0.00025712 0.01561999
 0.11206823 0.0060794  0.00412983 0.00227181 0.00893391 0.00260823
 0.00588378 0.00333696 0.00756503 0.00345948 0.00680805 0.01321699
 0.01308941 0.00918298 0.01289911 0.01416893 0.00303741 0.00796225
 0.00680734 0.00625669 0.0068789  0.00212403 0.00797836 0.
 0.         0.00264341 0.00368773 0.         0.         0.01850503
 0.         0.         0.         0.01873206 0.         0.00257633
 0.00211039 0.00013985 0.         0.00385451 0.00127667 0.01248296
 0.00877806 0.00227031 0.         0.00280328 0.02395948 0.01183059
 0.01277308 0.01325188 0.00336882 0.00088103 0.00086115 0.00598466
 0.00687134 0.00165107 0.00750931 0.         0.00600139 0.00407042
 0.         0.00216816 0.         0.0047493  0.00071753 0.
 0.         0.05922163 0.02787606 0.00969562 0.         0.
 0.         0.         0.00286168 0.00233052 0.         0.
 0.         0.         0.01502522 0.003453   0.         0.
 0.00324779 0.01034038 0.         0

In [ ]:
#Hyperparameter tuning done for decision tree classifier
#Random search
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(5,30,5),'criterion':['gini','entropy']} #Total 120 combinations 10*6*2
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15,cv=5) #randomly choose 15 combinations
clf_random.fit(X_Train, Y_Train)
rand_parm=clf_random.best_params_
print(rand_parm) #results could be different and misleading
#Now do grid search
print("GridSearchCV-Decision tree")
clf_grid = GridSearchCV(clf,parameters)
clf_grid.fit(X_Train, Y_Train)
grid_parm=clf_grid.best_params_
print(grid_parm)



RandomizedSearchCV-Decision tree
{'min_samples_leaf': 50, 'max_depth': 15, 'criterion': 'entropy'}
GridSearchCV-Decision tree
{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 30}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
#Construct Decision Trees using the best parameters
clfr = DecisionTreeClassifier(**rand_parm)
clfg = DecisionTreeClassifier(**grid_parm)

clfr.fit(X_Train,Y_Train)
clfr_predict = clfr.predict(X_Test)
clfg.fit(X_Train,Y_Train)
clfg_predict = clfg.predict(X_Test)


In [ ]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testset) after 1st hypertuning randomized search for Decision Tree:{0:6f}".format(clfr.score(X_Test,Y_Test)))
print("accuracy Score (testset) after 1st hypertuning grid search for Decision Tree:{0:6f}".format(clfg.score(X_Test,Y_Test)))

print("Confusion Matrix after hypertuning for Random Search Decision Tree")
print(confusion_matrix(Y_Test,clfr_predict))
print("=== 1st Random Search Classification Report ===")
print(classification_report(Y_Test,clfr_predict))

print("Confusion Matrix after hypertuning for Grid Search Decision Tree")
print(confusion_matrix(Y_Test,clfg_predict))
print("=== 1st Grid Search Classification Report ===")
print(classification_report(Y_Test,clfg_predict))

clf_cv_score = cross_val_score(clfr, X_Train, Y_Train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')

accuracy Score (testset) after 1st hypertuning randomized search for Decision Tree:0.897430
accuracy Score (testset) after 1st hypertuning grid search for Decision Tree:0.877380
Confusion Matrix after hypertuning for Random Search Decision Tree
[[11427   993]
 [  332   166]]
=== 1st Random Search Classification Report ===
              precision    recall  f1-score   support

           0       0.97      0.92      0.95     12420
           1       0.14      0.33      0.20       498

    accuracy                           0.90     12918
   macro avg       0.56      0.63      0.57     12918
weighted avg       0.94      0.90      0.92     12918

Confusion Matrix after hypertuning for Grid Search Decision Tree
[[11162  1258]
 [  326   172]]
=== 1st Grid Search Classification Report ===
              precision    recall  f1-score   support

           0       0.97      0.90      0.93     12420
           1       0.12      0.35      0.18       498

    accuracy                           0.88

In [ ]:
#2nd Hyperparameter tuning done for decision tree classifier
#Random search
print("RandomizedSearchCV-Decision tree")
parameters2={'min_samples_leaf' : range(5,30,5),'max_depth': range(5,30,5),'criterion':['gini','entropy'], 'class_weight': ["balanced"]}
clf_random2 = RandomizedSearchCV(clf,parameters2,n_iter=15,cv=5)
clf_random2.fit(X_Train, Y_Train)
rand_parm2=clf_random2.best_params_
print(rand_parm2) #results could be different and misleading
#Now do grid search
print("GridSearchCV-Decision tree")
clf_grid2 = GridSearchCV(clf,parameters2)
clf_grid2.fit(X_Train, Y_Train)
grid_parm2=clf_grid2.best_params_
print(grid_parm2)


RandomizedSearchCV-Decision tree
{'min_samples_leaf': 10, 'max_depth': 25, 'criterion': 'entropy', 'class_weight': 'balanced'}
GridSearchCV-Decision tree
{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 25, 'min_samples_leaf': 5}


In [ ]:
#2nd HyperParameterTuning
clfr2 = DecisionTreeClassifier(**rand_parm2)
clfg2 = DecisionTreeClassifier(**grid_parm2)

clfr2.fit(X_Train,Y_Train)
clfr2_predict = clfr2.predict(X_Test)
clfg2.fit(X_Train,Y_Train)
clfg2_predict = clfg2.predict(X_Test)


In [ ]:
print("==============2nd===============")
print("accuracy Score (testset) after 2nd hypertuning randomized search for Decision Tree:{0:6f}".format(clfr2.score(X_Test,Y_Test)))
print("accuracy Score (testset) after 2nd hypertuning grid search for Decision Tree:{0:6f}".format(clfg2.score(X_Test,Y_Test)))

print("Confusion Matrix after hypertuning for Random Search Decision Tree")
print(confusion_matrix(Y_Test,clfr2_predict))
print("=== 2nd Random Search Classification Report ===")
print(classification_report(Y_Test,clfr2_predict))

print("Confusion Matrix after hypertuning for Grid Search Decision Tree")
print(confusion_matrix(Y_Test,clfg2_predict))
print("=== 2nd Grid Search Classification Report ===")
print(classification_report(Y_Test,clfg2_predict))


==============2nd===============
accuracy Score (testset) after 2nd hypertuning randomized search for Decision Tree:0.771095
accuracy Score (testset) after 2nd hypertuning grid search for Decision Tree:0.797647
Confusion Matrix after hypertuning for Random Search Decision Tree
[[9501 2919]
 [  38  460]]
=== 2nd Random Search Classification Report ===
              precision    recall  f1-score   support

           0       1.00      0.76      0.87     12420
           1       0.14      0.92      0.24       498

    accuracy                           0.77     12918
   macro avg       0.57      0.84      0.55     12918
weighted avg       0.96      0.77      0.84     12918

Confusion Matrix after hypertuning for Grid Search Decision Tree
[[9835 2585]
 [  29  469]]
=== 2nd Grid Search Classification Report ===
              precision    recall  f1-score   support

           0       1.00      0.79      0.88     12420
           1       0.15      0.94      0.26       498

    accuracy      

In [ ]:
#3rd Hyperparameter tuning done for decision tree classifier
#Random search
print("RandomizedSearchCV-Decision tree")
parameters3={'min_samples_leaf' : range(1,5,1),'max_depth': range(5,30,5),'criterion':['gini','entropy'], 'class_weight': ["balanced"] }
clf_random3 = RandomizedSearchCV(clf,parameters3,n_iter=15,cv=5)
clf_random3.fit(X_Train, Y_Train)
rand_parm3=clf_random3.best_params_
print(rand_parm3) #results could be different and misleading
#Now do grid search
print("GridSearchCV-Decision tree")
clf_grid3 = GridSearchCV(clf,parameters3)
clf_grid3.fit(X_Train, Y_Train)
grid_parm3=clf_grid3.best_params_
print(grid_parm3)


RandomizedSearchCV-Decision tree
{'min_samples_leaf': 1, 'max_depth': 25, 'criterion': 'entropy', 'class_weight': 'balanced'}
GridSearchCV-Decision tree
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 25, 'min_samples_leaf': 1}


In [ ]:
#3rd HyperParameterTuning
clfr3 = DecisionTreeClassifier(**rand_parm3)
clfg3 = DecisionTreeClassifier(**grid_parm3)

clfr3.fit(X_Train,Y_Train)
clfr3_predict = clfr3.predict(X_Test)
clfg3.fit(X_Train,Y_Train)
clfg3_predict = clfg3.predict(X_Test)

In [ ]:
print("==============3rd===============")
print("accuracy Score (testset) after 3rd hypertuning randomized search for Decision Tree:{0:6f}".format(clfr3.score(X_Test,Y_Test)))
print("accuracy Score (testset) after 3rd hypertuning grid search for Decision Tree:{0:6f}".format(clfg3.score(X_Test,Y_Test)))

print("Confusion Matrix after hypertuning for Random Search Decision Tree")
print(confusion_matrix(Y_Test,clfr3_predict))
print("=== 3rd Random Search Classification Report ===")
print(classification_report(Y_Test,clfr3_predict))

print("Confusion Matrix after hypertuning for Grid Search Decision Tree")
print(confusion_matrix(Y_Test,clfg3_predict))
print("=== 3rd Grid Search Classification Report ===")
print(classification_report(Y_Test,clfg3_predict))

==============3rd===============
accuracy Score (testset) after 3rd hypertuning randomized search for Decision Tree:0.879161
accuracy Score (testset) after 3rd hypertuning grid search for Decision Tree:0.869639
Confusion Matrix after hypertuning for Random Search Decision Tree
[[10907  1513]
 [   48   450]]
=== 3rd Random Search Classification Report ===
              precision    recall  f1-score   support

           0       1.00      0.88      0.93     12420
           1       0.23      0.90      0.37       498

    accuracy                           0.88     12918
   macro avg       0.61      0.89      0.65     12918
weighted avg       0.97      0.88      0.91     12918

Confusion Matrix after hypertuning for Grid Search Decision Tree
[[10778  1642]
 [   42   456]]
=== 3rd Grid Search Classification Report ===
              precision    recall  f1-score   support

           0       1.00      0.87      0.93     12420
           1       0.22      0.92      0.35       498

    accura

In [11]:
#Normal randomforest==============================================================================
#=================================================================================================
rand_parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(1,10,2),'max_features':[2,3,4],'n_estimators':[20,30,40]}
rfc = RandomForestClassifier()
rfc.fit(X_Train, Y_Train)
rfc_predict=rfc.predict(X_Test)
print("accuracy Score (testset) for RandomForest:{0:6f}".format(rfc.score(X_Test,Y_Test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(Y_Test,rfc_predict))
#do random search with cross-validation
rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=15,cv=5)
rfc_random.fit(X_Train, Y_Train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
#create new classifier using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_Train,Y_Train)
rfc_predict = rfc.predict(X_Test)
print("accuracy Score (testset) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_Test,Y_Test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(Y_Test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(Y_Test,rfc_predict))
rfc_cv_score = cross_val_score(rfc, X_Train, Y_Train)
print(rfc_cv_score)
print('\n')

accuracy Score (testset) for RandomForest:0.959514
Confusion Matrix for Random Forest:
[[11974   446]
 [   77   421]]
{'n_estimators': 30, 'min_samples_leaf': 80, 'max_features': 4, 'max_depth': 3}
accuracy Score (testset) after hypertuning for Random Forest:0.961449
Confusion Matrix after hypertuning for Random Forest:
[[12420     0]
 [  498     0]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     12420
           1       0.00      0.00      0.00       498

    accuracy                           0.96     12918
   macro avg       0.48      0.50      0.49     12918
weighted avg       0.92      0.96      0.94     12918

[0.86666667 0.86666667 0.86666667 0.86666667 0.86811352]




In [ ]:
#2nd Normal randomforest==============================================================================
#=================================================================================================
rand_parameters2={'max_depth':range(10,50,5),'max_features':[5,10,15],'n_estimators':[10,15,20],'class_weight':["balanced"], 'ccp_alpha':[0.0], 'max_samples':[200]}

#do random search with cross-validation
rfc2_random = RandomizedSearchCV(rfc,rand_parameters2,n_iter=15,cv=5)
rfc2_random.fit(X_Train, Y_Train)
grid_parm_rfc2=rfc2_random.best_params_
print(grid_parm_rfc2)
#create new classifier using the best parameters
rfc2= RandomForestClassifier(**grid_parm_rfc2)
rfc2.fit(X_Train,Y_Train)
rfc2_predict = rfc2.predict(X_Test)
print("accuracy Score (testset) after hypertuning for Random Forest:{0:6f}".format(rfc2.score(X_Test,Y_Test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(Y_Test,rfc2_predict))
print("=== 2nd Classification Report ===")
print(classification_report(Y_Test,rfc2_predict))
rfc2_cv_score = cross_val_score(rfc2, X_Train, Y_Train)
print(rfc2_cv_score)
print('\n')

{'n_estimators': 20, 'max_samples': 200, 'max_features': 5, 'max_depth': 45, 'class_weight': 'balanced', 'ccp_alpha': 0.0}
accuracy Score (testset) after hypertuning for Random Forest:0.960443
Confusion Matrix after hypertuning for Random Forest:
[[12398    22]
 [  489     9]]
=== 2nd Classification Report ===
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     12420
           1       0.29      0.02      0.03       498

    accuracy                           0.96     12918
   macro avg       0.63      0.51      0.51     12918
weighted avg       0.94      0.96      0.94     12918

[0.86666667 0.86833333 0.86666667 0.86666667 0.86811352]




In [12]:
#3rd Normal randomforest==============================================================================
#=================================================================================================

rand_parameters3={'max_depth':range(10,100,20),'max_features':[5,10,15],'n_estimators':[10,20,30], 'max_samples':[200]}

#do random search with cross-validation
rfc3_random = RandomizedSearchCV(rfc,rand_parameters3,n_iter=15,cv=5)
rfc3_random.fit(X_Train, Y_Train)
grid_parm_rfc3=rfc3_random.best_params_
print(grid_parm_rfc3)
#create new classifier using the best parameters
rfc3= RandomForestClassifier(**grid_parm_rfc3)
rfc3.fit(X_Train,Y_Train)
rfc3_predict = rfc3.predict(X_Test)
print("accuracy Score (testset) after hypertuning for Random Forest:{0:6f}".format(rfc3.score(X_Test,Y_Test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(Y_Test,rfc3_predict))
print("=== 3rd Classification Report ===")
print(classification_report(Y_Test,rfc3_predict))
rfc3_cv_score = cross_val_score(rfc3, X_Train, Y_Train)
print(rfc3_cv_score)
print('\n')

{'n_estimators': 20, 'max_samples': 200, 'max_features': 10, 'max_depth': 50}
accuracy Score (testset) after hypertuning for Random Forest:0.959049
Confusion Matrix after hypertuning for Random Forest:
[[12378    42]
 [  487    11]]
=== 3rd Classification Report ===
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     12420
           1       0.21      0.02      0.04       498

    accuracy                           0.96     12918
   macro avg       0.58      0.51      0.51     12918
weighted avg       0.93      0.96      0.94     12918

[0.86       0.86833333 0.86833333 0.86666667 0.86978297]


